In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import DataCollatorWithPadding
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ConfiguredMetric:
    def __init__(self, metric, *metric_args, **metric_kwargs):
        self.metric = metric
        self.metric_args = metric_args
        self.metric_kwargs = metric_kwargs
    
    def add(self, *args, **kwargs):
        return self.metric.add(*args, **kwargs)
    
    def add_batch(self, *args, **kwargs):
        return self.metric.add_batch(*args, **kwargs)

    def compute(self, *args, **kwargs):
        return self.metric.compute(*args, *self.metric_args, **kwargs, **self.metric_kwargs)

    @property
    def name(self):
        return self.metric.name

    def _feature_names(self):
        return self.metric._feature_names()

In [3]:

df=pd.read_csv("datasetprime.csv")
df.rename(columns = {'UserID':'label',}, inplace = True)

df.head()

""""
import pandas as pd
import mysql.connector

# Database connection parameters
host = "your_host"
port = 3306  # default MySQL port
user = "your_username"
password = "your_password"
database = "your_database_name"

# Establishing the connection
conn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database)

# SQL query
query = "SELECT column1, column2, column3 FROM your_table_name"

# Fetching data into DataFrame
df = pd.read_sql(query, conn)

# Renaming columns if necessary
df.rename(columns={'Participants': 'label'}, inplace=True)

# Display the first few rows of the DataFrame
print(df.head())

# Close the connection
conn.close()
"""

'"\nimport pandas as pd\nimport mysql.connector\n\n# Database connection parameters\nhost = "your_host"\nport = 3306  # default MySQL port\nuser = "your_username"\npassword = "your_password"\ndatabase = "your_database_name"\n\n# Establishing the connection\nconn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database)\n\n# SQL query\nquery = "SELECT column1, column2, column3 FROM your_table_name"\n\n# Fetching data into DataFrame\ndf = pd.read_sql(query, conn)\n\n# Renaming columns if necessary\ndf.rename(columns={\'Participants\': \'label\'}, inplace=True)\n\n# Display the first few rows of the DataFrame\nprint(df.head())\n\n# Close the connection\nconn.close()\n'

In [4]:
def combine_columns_to_text(df, exclude_columns=None, column_weights=None):
    if exclude_columns is None:
        exclude_columns = []
    if column_weights is None:
        column_weights = {}
    if df.empty:
        return pd.DataFrame(columns=['text', 'label'])
    
    # Store the 'label' column if it exists and is not in the exclude list
    label_col = df['label'] if 'label' in df.columns and 'label' not in exclude_columns else None
    
    # Drop the columns that need to be excluded
    df_combined = df.drop(columns=exclude_columns, errors='ignore')
    
    # Apply weighting to specified columns by repeating their contents along with the column name
    for column, weight in column_weights.items():
        if column in df_combined.columns:
            df_combined[column] = df_combined[column].apply(
                lambda x: (' '.join([f"{column}:{x}"] * weight)) if pd.notnull(x) else ''
            )
    
    # Combine all columns into a single column
    df_combined['text'] = df_combined.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    
    # Include the 'label' column in the result if it was stored
    if label_col is not None:
        df_combined['label'] = label_col
    
    # Return the DataFrame with the 'text' and 'label' columns
    return df_combined[['text', 'label']] if label_col is not None else df_combined[['text']]


In [5]:
# Define weights for each column
column_weights = {
    'Username': 0,
    'Email': 0,
    'Gender': 0,
    'Age': 0,
    'Heig   ht': 0,
    'Weight': 0,
    'ActivityLevel': 15,
    'GoalID': 10,
    'DietPreferenceID': 15,
    'RegionID': 15,
    'AllergenID': 30  # Higher weight for AllergenID
}

# Combine columns with specified weights
generalDataframe = combine_columns_to_text(df, exclude_columns=['DietID'], column_weights=column_weights)


In [6]:
generalDataframe.head()

,text,label
0,1 False ActivityLevel:Sedentary Activity...,1
1,2 True ActivityLevel:Very Active Activit...,2
2,3 True ActivityLevel:Active ActivityLeve...,3
3,4 False ActivityLevel:Sedentary Activity...,4
4,5 False ActivityLevel:Active ActivityLev...,5


In [7]:

# Encode labels
label_encoder = LabelEncoder()
generalDataframe['label'] = label_encoder.fit_transform(generalDataframe['label'])

# Split dataset
train_df, val_df = train_test_split(generalDataframe, test_size=0.001, random_state=42)


In [8]:
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"])

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

In [9]:
numberOfClasses=max(train_df['label'])

In [10]:
myDataset=Dataset.from_pandas(train_df)

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=numberOfClasses+1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.we

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Training
trainer.train()

# Evaluation
eval_results = trainer.evaluate(tokenized_val_dataset)
print(eval_results)


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 3/9366 [00:32<27:32:44, 10.59s/it]

KeyboardInterrupt: 

In [ ]:
import torch
def predict_label(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probabilities, dim=-1).item()
    return label_encoder.inverse_transform([predicted_label])[0]

In [ ]:
# Example user information with column names
text_to_predict = """
    Username:JohnDoe Email:johndoe@example.com Gender:Male Age:90 Height:180 Weight:80 
    ActivityLevel:Sedentary GoalID:10 DietPreferenceID:2 RegionID:8 AllergenID:10
    """

# Predict the label for the input text
predicted_label = predict_label(text_to_predict)

# Print the predicted label
print(f"The predicted label for the input text is: {predicted_label}")


The predicted label for the input text is: 668


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(generalDataframe['text'], generalDataframe['label'], random_state=123)


In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', strip_accents='unicode', lowercase=True)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)


In [ ]:
modelNb=MultinomialNB()
modelNb.fit(X_train_transformed,X_train)
predictions=modelNb.predict(X_test_transformed)

AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'predict'

In [ ]:
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train_transformed, X_train)
svm_predictions = svm_model_linear.predict(X_test_transformed)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 12).fit(X_train_transformed, X_train)
knn_predictions = knn.predict(X_test_transformed) 

In [ ]:

def predict_label_id_multinomial_nb(text):
    text_transformed = vectorizer.transform([text])
    return modelNb.predict(text_transformed)[0]

def predict_label_id_svm(text):
    text_transformed = vectorizer.transform([text])
    return svm_model_linear.predict(text_transformed)[0]

def predict_label_id_knn(text):
    text_transformed = vectorizer.transform([text])
    return knn.predict(text_transformed)[0]


In [ ]:
text_to_predict = """
    Username:JohnDoe Email:johndoe@example.com Gender:Male Age:90 Height:180 Weight:80 
    ActivityLevel:Sedentary GoalID:10 DietPreferenceID:2 RegionID:10 AllergenID:10
    """

# Predict using Logistic Regression

# Predict using Multinomial Naive Bayes
mnb_label_id = predict_label_id_multinomial_nb(text_to_predict)
print(f"Multinomial Naive Bayes predicted label ID: {mnb_label_id}")

# Predict using SVM
svm_label_id = predict_label_id_svm(text_to_predict)
print(f"SVM predicted label ID: {svm_label_id}")

# Predict using KNN
knn_label_id = predict_label_id_knn(text_to_predict)
print(f"KNN predicted label ID: {knn_label_id}")

Multinomial Naive Bayes predicted label ID: 465 True       ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 RegionID:9 AllergenID:10 AllergenID:10 All